In [1]:
import requests
import time
import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
sys.path.append(project_folder_path)
from utils.db_functs import *

/home/r2f/GitHubApps/cev-data-lake


## 1) Create Database

### 1.1) Set Database file parameters
Database file is a sqlite .db file

In [2]:
version = 'v1'
db_file_name = f'cev-database-reports-{version}.db'
print(db_file_name)
db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', db_file_name)
db_file_path

cev-database-reports-v1.db


'/home/r2f/GitHubApps/cev-data-lake/data/sqlite/cev-database-reports-v1.db'

### 1.2) Create sqlite database file

In [3]:
create_database(db_file_path)

Database file '/home/r2f/GitHubApps/cev-data-lake/data/sqlite/cev-database-reports-v1.db' created successfully.


## 2) Create Tables
Database tables are:
* `regiones`
* `comunas`
* `viewstate_region`
* `html_files_by_comuna_and_search`
* `html_files_by_comuna_page_and_search`

### 2.1) Define all tables in a dicitonary
1. Dictionay key are the tables' names
2. Dictionary values are the tables queries

In [4]:
table_queries = {
    'regiones': """
        CREATE TABLE regiones (
            region_id INTEGER PRIMARY KEY,
            region_name TEXT,
            UNIQUE (region_id, region_name)
        )
    """,
    'comunas': """
        CREATE TABLE comunas (
            comuna_id INTEGER PRIMARY KEY,
            comuna_name TEXT,
            region_id INTEGER,
            FOREIGN KEY(region_id) REFERENCES regiones(region_id),
            UNIQUE(comuna_id, comuna_name, region_id)
        )
    """,    
    'evals_summary': """
        CREATE TABLE evals_summary (
            eval_id TEXT PRIMARY KEY,
            comuna_id INTEGER,
            region_id INTEGER,
            tipo_evaluacion INTEGER,
            identificacion_vivienda TEXT,
            tipologia TEXT,
            comuna TEXT,
            proyecto TEXT,
            CE TEXT,
            CEE TEXT,                      
            FOREIGN KEY(comuna_id) REFERENCES comunas(comuna_id),
            FOREIGN KEY(region_id) REFERENCES regiones(region_id)
        )
    """,
    'reports_summary': """
        CREATE TABLE reports_summary (
            eval_id TEXT PRIMARY KEY,
            comuna_id INTEGER,
            region_id INTEGER,
            tipo_evaluacion INTEGER,
            version_evaluacion INTEGER,
            codigo_evaluacion TEXT,
            pdf_file_name TEXT,
            is_pdf_file_valid INTEGER,                                
            FOREIGN KEY(comuna_id) REFERENCES comunas(comuna_id),
            FOREIGN KEY(region_id) REFERENCES regiones(region_id)
        )
    """,
    'informe_v2_pagina1': """
        CREATE TABLE informe_v2_pagina1 (
            eval_id TEXT PRIMARY KEY,
            tipo_evaluacion TEXT,
            codigo_evaluacion TEXT,
            region TEXT,
            comuna TEXT,
            direccion TEXT,
            rol_vivienda_proyecto TEXT,
            tipo_vivienda TEXT,
            superficie_interior_util_m2 TEXT,
            porcentaje_ahorro TEXT,
            letra_eficiencia_energetica_dem TEXT,
            demanda_calefaccion_kwh_m2_ano TEXT,
            demanda_enfriamiento_kwh_m2_ano TEXT,
            demanda_total_kwh_m2_ano TEXT,
            emitida_el TEXT
        )
    """,
    'informe_v2_pagina2': """
        CREATE TABLE informe_v2_pagina2 (
            eval_id TEXT PRIMARY KEY,
            region TEXT,
            comuna TEXT,
            direccion TEXT,
            rol_vivienda TEXT,
            tipo_vivienda TEXT,
            zona_termica TEXT,
            superficie_interior_util_m2 TEXT,
            solicitado_por TEXT,
            evaluado_por TEXT,
            codigo_evaluacion TEXT,
            demanda_calefaccion_kwh_m2_ano TEXT,
            demanda_enfriamiento_kwh_m2_ano TEXT,
            demanda_total_kwh_m2_ano TEXT,
            demanda_total_bis_kwh_m2_ano TEXT,
            demanda_total_referencia_kwh_m2_ano TEXT,
            porcentaje_ahorro TEXT,
            muro_principal_descripcion TEXT,
            muro_principal_exigencia_W_m2_K TEXT,
            muro_secundario_descripcion TEXT,
            muro_secundario_exigencia_W_m2_K TEXT,
            piso_principal_descripcion TEXT,
            piso_principal_exigencia_W_m2_K TEXT,
            puerta_principal_descripcion TEXT,
            puerta_principal_exigencia_W_m2_K TEXT,
            techo_principal_descripcion TEXT,
            techo_principal_exigencia_W_m2_K TEXT,
            techo_secundario_descripcion TEXT,
            techo_secundario_exigencia_W_m2_K TEXT,
            superficie_vidriada_principal_descripcion TEXT,
            superficie_vidriada_principal_exigencia TEXT,
            superficie_vidriada_secundaria_descripcion TEXT,
            superficie_vidriada_secundaria_exigencia TEXT,
            ventilacion_rah_descripcion TEXT,
            ventilacion_rah_exigencia TEXT,
            infiltraciones_rah_descripcion TEXT,
            infiltraciones_rah_exigencia TEXT
        )
    """,
    'informe_v2_pagina3_consumos': """
        CREATE TABLE informe_v2_pagina3_consumos (
            eval_id TEXT PRIMARY KEY,
            codigo_evaluacion TEXT,
            agua_caliente_sanitaria_kwh_m2 TEXT,
            agua_caliente_sanitaria_perc TEXT,
            iluminacion_kwh_m2 TEXT,
            iluminacion_per TEXT,
            calefaccion_kwh_m2 TEXT,
            calefaccion_kwh_per TEXT,
            energia_renovable_no_convencional_kwh_m2 TEXT,
            energia_renovable_no_convencional_per TEXT,
            consumo_total_kwh_m2 TEXT,
            emisiones_kgco2_m2_ano TEXT,
            calefaccion_descripcion_proy TEXT,
            calefaccion_consumo_proy_kwh TEXT,
            calefaccion_consumo_proy_per TEXT,
            iluminacion_descripcion_proy TEXT,
            iluminacion_consumo_proy_kwh TEXT,
            iluminacion_consumo_proy_per TEXT,
            agua_caliente_sanitaria_descripcion_proy TEXT,
            agua_caliente_sanitaria_consumo_proy_kwh TEXT,
            agua_caliente_sanitaria_consumo_proy_per TEXT,
            energia_renovable_no_convencional_descripcion_proy TEXT,
            energia_renovable_no_convencional_consumo_proy_kwh TEXT,
            energia_renovable_no_convencional_consumo_proy_per TEXT,
            consumo_total_requerido_proy_kwh TEXT,
            calefaccion_descripcion_ref TEXT,
            calefaccion_consumo_ref_kwh TEXT,
            calefaccion_consumo_ref_per TEXT,
            iluminacion_descripcion_ref TEXT,
            iluminacion_consumo_ref_kwh TEXT,
            iluminacion_consumo_ref_per TEXT,
            agua_caliente_sanitaria_descripcion_ref TEXT,
            agua_caliente_sanitaria_consumo_ref_kwh TEXT,
            agua_caliente_sanitaria_consumo_ref_per TEXT,
            energia_renovable_no_convencional_descripcion_ref TEXT,
            energia_renovable_no_convencional_consumo_ref_kwh TEXT,
            energia_renovable_no_convencional_consumo_ref_per TEXT,
            consumo_total_requerido_ref_kwh TEXT,
            consumo_ep_calefaccion_kwh TEXT,
            consumo_ep_agua_caliente_sanitaria_kwh TEXT,
            consumo_ep_iluminacion_kwh TEXT,
            consumo_ep_ventiladores_kwh TEXT,
            generacion_ep_fotovoltaicos_kwh TEXT,
            aporte_fotovoltaicos_consumos_basicos_kwh TEXT,
            diferencia_fotovoltaica_para_consumo_kwh TEXT,
            aporte_solar_termica_consumos_basicos_kwh TEXT,
            aporte_solar_termica_agua_caliente_sanitaria_kwh TEXT,
            total_consumo_ep_antes_fotovoltaica_kwh TEXT,
            aporte_fotovoltaicos_consumos_basicos_kwh_bis TEXT,
            consumos_basicos_a_suplir_kwh TEXT,
            consumo_total_ep_obj_kwh TEXT,
            consumo_total_ep_ref_kwh TEXT,
            coeficiente_energetico_c TEXT
        )
    """,
    'informe_v2_pagina3_envolvente': """
        CREATE TABLE informe_v2_pagina3_envolvente (
            eval_id TEXT PRIMARY KEY,
            codigo_evaluacion TEXT,
            orientacion TEXT,
            elementos_opacos_area_m2 TEXT,
            elementos_opacos_U_W_m2_K TEXT,
            elementos_traslucidos_area_m2 TEXT,
            elementos_traslucidos_U_W_m2_K TEXT,
            P01_W_K TEXT,
            P02_W_K TEXT,
            P03_W_K TEXT,
            P04_W_K TEXT,
            P05_W_K TEXT,
            UA_phiL TEXT
        )
    """,
    'informe_v2_pagina4': """
        CREATE TABLE informe_v2_pagina4 (
            eval_id TEXT PRIMARY KEY,
            codigo_evaluacion TEXT,
            mes_id TEXT,
            demanda_calef_viv_eval_kwh TEXT,
            demanda_calef_viv_eval_comment TEXT,
            demanda_calef_viv_ref_kwh TEXT,
            demanda_calef_viv_ref_comment TEXT,
            demanda_enfri_viv_eval_kwh TEXT,
            demanda_enfri_viv_eval_comment TEXT,
            demanda_enfri_viv_ref_kwh TEXT,
            demanda_enfri_viv_ref_comment TEXT,
            sobrecalentamiento_viv_eval_hr TEXT,
            sobrecalentamiento_viv_eval_comment TEXT,
            sobrecalentamiento_viv_ref_hr TEXT,
            sobrecalentamiento_viv_ref_comment TEXT,
            sobreenfriamiento_viv_eval_hr TEXT,
            sobreenfriamiento_viv_eval_comment TEXT,
            sobreenfriamiento_viv_ref_hr TEXT,
            sobreenfriamiento_viv_ref_comment TEXT
        )
    """,
    'informe_v2_pagina5': """
        CREATE TABLE informe_v2_pagina5 (
            eval_id TEXT PRIMARY KEY,
            codigo_evaluacion TEXT,
            content TEXT
        )
    """,
    'informe_v2_pagina6': """
        CREATE TABLE informe_v2_pagina6 (
            eval_id TEXT PRIMARY KEY,
            codigo_evaluacion TEXT,
            content TEXT
        )
    """,
    'informe_v2_pagina7': """
        CREATE TABLE informe_v2_pagina7 (
            eval_id TEXT PRIMARY KEY,
            codigo_evaluacion TEXT,
            mandante_nombre TEXT,
            mandante_rut TEXT,
            evaluador_nombre TEXT,
            evaluador_rut TEXT,
            evaluador_rol_minvu TEXT
        )
    """
    
    
}

### 2.2) Create Tables

In [5]:
create_tables(db_file_path, table_queries)

Table 'regiones' created successfully.
Table 'comunas' created successfully.
Table 'evals_summary' created successfully.
Table 'reports_summary' created successfully.
Table 'informe_v2_pagina1' created successfully.
Table 'informe_v2_pagina2' created successfully.
Table 'informe_v2_pagina3_consumos' created successfully.
Table 'informe_v2_pagina3_envolvente' created successfully.
Table 'informe_v2_pagina4' created successfully.
Table 'informe_v2_pagina5' created successfully.
Table 'informe_v2_pagina6' created successfully.
Table 'informe_v2_pagina7' created successfully.


### 2.3) Populating Tables

#### 2.3.1) Tabla `Regiones`

In [6]:
# Region names and corresponding Ids    
with open(os.path.join(project_folder_path, 'utils/json_files/region_name__region_id.json')) as json_file:
    region_name__region_id_dict = json.load(json_file)
    
# Invert the dictionary
region_id__region_name_dict = {value: key for key, value in region_name__region_id_dict.items()}

# Create dataframe from dictionary
regiones_df = pd.DataFrame(list(region_id__region_name_dict.items()), columns=['region_id', 'region_name'])
regiones_df['region_id'] = regiones_df['region_id'].astype(int)

In [7]:
# Fill table Regiones
fill_table_from_dataframe(db_file_path, 'regiones', regiones_df)

Table 'regiones' filled successfully.


#### 2.3.2) Tabla `Comunas`

In [8]:
# Commune Id and corresponding Commune Name       
with open(os.path.join(project_folder_path, 'utils/json_files/commune_id__commune_name.json')) as json_file:
    commune_id__commune_name_dict = json.load(json_file)
# Create dataframe from dictionary
comunas_df = pd.DataFrame(list(commune_id__commune_name_dict.items()), columns=['comuna_id', 'comuna_name'])
comunas_df['comuna_id'] = comunas_df['comuna_id'].astype(int)
comunas_df.head()  

,comuna_id,comuna_name
0,1,Algarrobo
1,2,Alhué
2,3,Alto Biobío
3,4,Alto del Carmen
4,5,Alto Hospicio


In [9]:
# Region Id and corresponding Communes Ids       
with open(os.path.join(project_folder_path, 'utils/json_files/region_id__commune_id.json')) as json_file:
    region_id__commune_id_dict = json.load(json_file)
# Transform the dictionary 'region_id__commune_id_dict' into a list of dictionaries
data_list = []
for key, values in region_id__commune_id_dict.items():
    for value in values:
        data_list.append({'comuna_id': value, 'region_id': key})
# Create a DataFrame from the list of dictionaries
comuna_region_df = pd.DataFrame(data_list)
comuna_region_df['comuna_id'] = comuna_region_df['comuna_id'].astype(int)
comuna_region_df['region_id'] = comuna_region_df['region_id'].astype(int)
# Add region_id column to dataframe
comunas_df = pd.merge(comunas_df, comuna_region_df, on='comuna_id', how='left')
comunas_df.head()

,comuna_id,comuna_name,region_id
0,1,Algarrobo,5
1,2,Alhué,13
2,3,Alto Biobío,8
3,4,Alto del Carmen,3
4,5,Alto Hospicio,1


In [10]:
# Fill table Comunas
fill_table_from_dataframe(db_file_path, 'comunas', comunas_df)

Table 'comunas' filled successfully.


**Remaining tables are filled in next steps !!!**

## END